In [5]:
import torch
from models.experimental import attempt_load
from utils.torch_utils import select_device

# Load the YOLOv5 PyTorch model
weights_path = "D:/documents/zhonglujiaoke/datasets/YOLO/yolov5_prediction/yolov5_prediction/weights/best.pt"
device = select_device("")  # Use an empty string to use the default device
model = attempt_load(weights_path, map_location=device)

# Set the model to evaluation mode
model.eval()

# Create dummy input data
dummy_input = torch.randn(1, 3, 640, 640).to(device)

# Export the model to ONNX format
onnx_path = "yolov5.onnx"
torch.onnx.export(model, dummy_input, onnx_path, opset_version=11)


Fusing layers... 
D:\documents\zhonglujiaoke\datasets\YOLO\yolov5_prediction\yolov5_prediction\models\yolo.py:207: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if augment:
D:\documents\zhonglujiaoke\datasets\YOLO\yolov5_prediction\yolov5_prediction\models\yolo.py:119: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if profile:
D:\documents\zhonglujiaoke\datasets\YOLO\yolov5_prediction\yolov5_prediction\models\yolo.py:123: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python

In [18]:
import tensorflow as tf
import tf2onnx

# Load the ONNX model
onnx_path = "yolov5.onnx"
onnx_model = onnx.load(onnx_path)

# Convert ONNX to TensorFlow GraphDef
tf_rep = tf2onnx.convert.from_onnx(onnx_model)

# Convert TensorFlow GraphDef to SavedModel
with tf.Graph().as_default() as tf_graph:
    tf.import_graph_def(tf_rep.graph.as_graph_def(), name='')

saved_model_path = "yolov5_saved_model"
tf.saved_model.save(tf.compat.v1.Session(graph=tf_graph), saved_model_path)

# Convert SavedModel to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
tflite_path = "yolov5.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)


AttributeError: module 'tf2onnx.convert' has no attribute 'from_onnx'

In [22]:
import torch
import cv2
import numpy as np
from yolo import Model
from utils.datasets import LoadImages
from utils.general import check_img_size, non_max_suppression, scale_coords

import os

model_weight_path = 'D:/documents/zhonglujiaoke/datasets/YOLO/yolov5_prediction/yolov5_prediction/weights/best.pt'
normalized_path = os.path.normpath(model_weight_path)

print(normalized_path)


# Load the image
image_path = 'D:\documents\zhonglujiaoke\datasets\YOLO\yolov5_prediction\yolov5_prediction\IMG_5859.jpg'
img = cv2.imread(image_path)

# Preprocess the image
img = cv2.resize(img, (640, 640))
img = np.array(img).transpose((2, 0, 1))
img = np.expand_dims(img, axis=0)

# Make predictions
pred = model(img)[0]

# Postprocess the predictions
pred = non_max_suppression(pred, conf_thres=0.5, iou_thres=0.4)[0]

# Draw bounding boxes around the detected defects
for det in pred:
    x1, y1, x2, y2, conf, cls = det
    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
    cv2.putText(img, f'Pothole: {conf:.2f}', (int(x1), int(y1)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

# Display the image
cv2.imshow('Image', img)
cv2.waitKey(0)


D:\documents\zhonglujiaoke\datasets\YOLO\yolov5_prediction\yolov5_prediction\weights\best.pt


TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray